In [7]:
import os
from Bio import SeqIO

In [2]:
def read_data():
    base_directory = './e_coli_paidb/'
    data = {}
    
    for name in os.listdir(base_directory):
        dir_path = os.path.join(base_directory, name)

        if os.path.isdir(dir_path):
            fasta_file = os.path.join(dir_path, f"{name}.fasta")
            pai_file   = os.path.join(dir_path, f"{name}.pai")
            cpai_file  = os.path.join(dir_path, f"{name}.cpai")
            npai_file  = os.path.join(dir_path, f"{name}.npai")

            data[name] = {}
            genome_sequence = next(SeqIO.parse(fasta_file, 'fasta'))
            data[name]['genome_sequence'] = genome_sequence

            with open(pai_file, 'r') as file:
                data[name]['pai_sequences'] = []

                while True:
                    line = file.readline()
                    if not line:
                        break

                    line = line.split()
                    pai_start_index = int(line[0])
                    pai_end_index = int(line[1])

                    pai_sequence = genome_sequence.seq[pai_start_index:pai_end_index]

                    data[name]['pai_sequences'].append((pai_sequence, (pai_start_index, pai_end_index)))

            with open(cpai_file, 'r') as file:
                data[name]['cpai_sequences'] = []

                while True:
                    line = file.readline()
                    if not line:
                        break

                    line = line.split()
                    cpai_start_index = int(line[0])
                    cpai_end_index = int(line[1])

                    cpai_sequence = genome_sequence.seq[cpai_start_index:cpai_end_index]

                    data[name]['cpai_sequences'].append((cpai_sequence, (cpai_start_index, cpai_end_index)))

            with open(npai_file, 'r') as file:
                data[name]['npai_sequences'] = []

                while True:
                    line = file.readline()
                    if not line:
                        break

                    line = line.split()
                    npai_start_index = int(line[0])
                    npai_end_index = int(line[1])

                    npai_sequence = genome_sequence.seq[npai_start_index:npai_end_index]

                    data[name]['npai_sequences'].append((npai_sequence, (npai_start_index, npai_end_index)))

    return data

### TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(sequence, k):
    kmers = get_kmers(str(sequence), k)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([kmers])
    tfidf_matrix = X.toarray()
    return tfidf_matrix, vectorizer

def get_kmers(sequence, k):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return ' '.join(kmers)

def get_kmers_map(tfidf_matrix, vectorizer, sequence, top_n=20):
    feature_names = vectorizer.get_feature_names_out()
    feature_scores = tfidf_matrix.flatten()
    
    top_kmers_indices = feature_scores.argsort()[-top_n:][::-1]
    top_kmers = [(feature_names[i], feature_scores[i]) for i in top_kmers_indices]

    mapped_kmers = {}
    for kmer, score in top_kmers:
        kmer = kmer.upper()
        count = sequence.count(kmer)
        mapped_kmers[kmer] = {'score': score, 'count': count}
        
    return mapped_kmers

def find_unique_kmers(full_sequence, start_index, end_index, significant_kmers, unique_kmers):
    for kmer, details in significant_kmers.items():
        left_occurrences = full_sequence[:start_index].count(kmer)
        right_occurrences = full_sequence[end_index:].count(kmer)
        occurrences = left_occurrences + right_occurrences
        if occurrences == 0:
            unique_kmers.append((kmer, details['count']))

In [3]:
def get_unique_patterns(full_sequence, island_sequence, start_index, end_index, k_max):
    
    unique_kmers = []
    
    for k in range(4, k_max):
        tfidf_matrix, vectorizer = tfidf(island_sequence, k)
        kmers = get_kmers_map(tfidf_matrix, vectorizer, island_sequence)
        find_unique_kmers(full_sequence, start_index, end_index, kmers, unique_kmers)
        
    return unique_kmers

In [39]:
e_coli_data = read_data()

In [5]:
import logging

logging.basicConfig(filename='tfidf.log', level=logging.INFO,
                    format='%(message)s')

console = logging.StreamHandler()
console.setLevel(logging.INFO)
console.setFormatter(logging.Formatter('%(message)s'))

logging.getLogger('').addHandler(console)

In [6]:
unique_pai_patterns  = {}
unique_cpai_patterns = {}
unique_npai_patterns = {}

k_max = 100
logging.info(f'Max pattern length: {k_max}')
logging.info('--------------------------')

for i, (name, data) in enumerate(e_coli_data.items()):
    logging.info(f'Genome {i+1}/{len(e_coli_data)}: {name}')
    
    if len(data['pai_sequences']) != 0:
        unique_pai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (pai_sequence, indices) in enumerate(data['pai_sequences']):
            unique_patterns = get_unique_patterns(genome_sequence, pai_sequence, indices[0], indices[1], k_max)

            unique_pai_patterns[name]['pai_sequence'] = pai_sequence
            unique_pai_patterns[name]['indices'] = indices
            unique_pai_patterns[name]['patterns'] = unique_patterns

            logging.info(f"PAI {j+1}/{len(data['pai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'e_coli_paidb/{name}/patterns/pai/', exist_ok=True)
            
            with open(f'e_coli_paidb/{name}/patterns/pai/pai_{j}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
                        
    else:
        logging.info('No PAI sequences')
    
    if len(data['cpai_sequences']) != 0:    
        unique_cpai_patterns[name] = {}

        genome_sequence = data['genome_sequence']
        for j, (cpai_sequence, indices) in enumerate(data['cpai_sequences']):
            unique_patterns = get_unique_patterns(genome_sequence, cpai_sequence, indices[0], indices[1], k_max)

            unique_cpai_patterns[name]['cpai_sequence'] = cpai_sequence
            unique_cpai_patterns[name]['indices'] = indices
            unique_cpai_patterns[name]['patterns'] = unique_patterns
            
            logging.info(f"CPAI {j+1}/{len(data['cpai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'e_coli_paidb/{name}/patterns/cpai/', exist_ok=True)
            
            with open(f'e_coli_paidb/{name}/patterns/cpai/cpai_{j}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
            
    else:
        logging.info('No CPAI sequences')
        
    if len(data['npai_sequences']) != 0:
        unique_npai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (npai_sequence, indices) in enumerate(data['npai_sequences']):
            unique_patterns = get_unique_patterns(genome_sequence, npai_sequence, indices[0], indices[1], k_max)

            unique_npai_patterns[name]['npai_sequence'] = npai_sequence
            unique_npai_patterns[name]['indices'] = indices
            unique_npai_patterns[name]['patterns'] = unique_patterns

            logging.info(f"NPAI {j+1}/{len(data['npai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'e_coli_paidb/{name}/patterns/npai/', exist_ok=True)
            
            with open(f'e_coli_paidb/{name}/patterns/npai/npai_{j}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
    
    else:
        logging.info('No NPAI sequences')
    logging.info('--------------------------')

Max pattern length: 100
--------------------------
Genome 1/90: NC_010468
No PAI sequences
CPAI 1/3: 1742 unique patterns
CPAI 2/3: 1735 unique patterns
CPAI 3/3: 631 unique patterns
NPAI 1/2: 1743 unique patterns
NPAI 2/2: 1746 unique patterns
--------------------------
Genome 2/90: NC_011748
No PAI sequences
CPAI 1/10: 641 unique patterns
CPAI 2/10: 1760 unique patterns
CPAI 3/10: 1751 unique patterns
CPAI 4/10: 1749 unique patterns
CPAI 5/10: 1619 unique patterns
CPAI 6/10: 581 unique patterns
CPAI 7/10: 36 unique patterns
CPAI 8/10: 1638 unique patterns
CPAI 9/10: 1648 unique patterns
CPAI 10/10: 1066 unique patterns
NPAI 1/1: 1740 unique patterns
--------------------------
Genome 3/90: NC_011751
PAI 1/1: 1454 unique patterns
CPAI 1/7: 657 unique patterns
CPAI 2/7: 1754 unique patterns
CPAI 3/7: 1738 unique patterns
CPAI 4/7: 1745 unique patterns
CPAI 5/7: 1753 unique patterns
CPAI 6/7: 1748 unique patterns
CPAI 7/7: 1399 unique patterns
NPAI 1/2: 1748 unique patterns
NPAI 2/2: 173

CPAI 3/8: 1752 unique patterns
CPAI 4/8: 1749 unique patterns
CPAI 5/8: 1744 unique patterns
CPAI 6/8: 1736 unique patterns
CPAI 7/8: 1743 unique patterns
CPAI 8/8: 1746 unique patterns
NPAI 1/3: 1749 unique patterns
NPAI 2/3: 1749 unique patterns
NPAI 3/3: 1750 unique patterns
--------------------------
Genome 34/90: NC_009800
No PAI sequences
CPAI 1/4: 654 unique patterns
CPAI 2/4: 1768 unique patterns
CPAI 3/4: 1407 unique patterns
CPAI 4/4: 1736 unique patterns
NPAI 1/1: 1688 unique patterns
--------------------------
Genome 35/90: NC_004431
PAI 1/2: 14 unique patterns
PAI 2/2: 4 unique patterns
CPAI 1/12: 664 unique patterns
CPAI 2/12: 1716 unique patterns
CPAI 3/12: 1744 unique patterns
CPAI 4/12: 1753 unique patterns
CPAI 5/12: 10 unique patterns
CPAI 6/12: 1742 unique patterns
CPAI 7/12: 1678 unique patterns
CPAI 8/12: 1626 unique patterns
CPAI 9/12: 1753 unique patterns
CPAI 10/12: 1748 unique patterns
CPAI 11/12: 1633 unique patterns
CPAI 12/12: 6 unique patterns
NPAI 1/2: 17

--------------------------
Genome 66/90: NC_012947
No PAI sequences
CPAI 1/5: 1756 unique patterns
CPAI 2/5: 1767 unique patterns
CPAI 3/5: 1742 unique patterns
CPAI 4/5: 1743 unique patterns
CPAI 5/5: 1753 unique patterns
NPAI 1/2: 0 unique patterns
NPAI 2/2: 1684 unique patterns
--------------------------
Genome 67/90: NC_011747
No PAI sequences
CPAI 1/3: 1803 unique patterns
CPAI 2/3: 1759 unique patterns
CPAI 3/3: 1803 unique patterns
NPAI 1/1: 1813 unique patterns
--------------------------
Genome 68/90: NC_013654
No PAI sequences
CPAI 1/4: 1746 unique patterns
CPAI 2/4: 1413 unique patterns
CPAI 3/4: 1737 unique patterns
CPAI 4/4: 1394 unique patterns
NPAI 1/2: 1745 unique patterns
NPAI 2/2: 1743 unique patterns
--------------------------
Genome 69/90: NC_011745
No PAI sequences
CPAI 1/11: 659 unique patterns
CPAI 2/11: 1561 unique patterns
CPAI 3/11: 1739 unique patterns
CPAI 4/11: 1528 unique patterns
CPAI 5/11: 1410 unique patterns
CPAI 6/11: 1751 unique patterns
CPAI 7/11: 13

In [7]:
unique_pai_patterns

{'NC_011751': {'pai_sequence': Seq('CCCGGATAGCTCAGTCGGTAGAGCAGGGGATTGAAAATCCCCGTGTCCTTGGTT...CAC'),
  'indices': (3445924, 3503263),
  'patterns': [('ACATTGGCATT', 3),
   ('GCATCAGAATA', 3),
   ('CAGATTCCCCCC', 2),
   ('TTCCCTGCCCGG', 2),
   ('AGCGGCACGGAT', 2),
   ('AGTGTCCTGCTG', 2),
   ('ATAATCCCTGTT', 2),
   ('GCCAAGGCCGGC', 2),
   ('TTCAAGAATTTTT', 2),
   ('TTTCCAGCATCAT', 2),
   ('TTCCATCAATAAC', 2),
   ('CCAGCGGCACGGA', 2),
   ('GCCAAGGCCGGCG', 2),
   ('ATTTCAGAATATA', 2),
   ('TCCACATTCTGTA', 2),
   ('GGGCTTCAGCCTG', 2),
   ('TCCACATCATCCG', 2),
   ('CAGGCTGATCAAA', 2),
   ('TTTCCGCTGTTTAC', 2),
   ('AGGGCTTCAGCCTG', 2),
   ('AATAATCCCTGTTG', 2),
   ('CAGGCTGATCAAATG', 2),
   ('TAAGAATAACATTTC', 2),
   ('TTTCTGCTCATCCAG', 2),
   ('AATAATCCCTGTTGA', 2),
   ('TGGCATCAGAATACTC', 2),
   ('CATGCTCATTGAAAAAATACT', 1),
   ('CATGGAACATCGTTCAGTGATT', 1),
   ('CATGCGTCACACCTCGTCCTTT', 1),
   ('CATGGACTCATTAACACCCTTT', 1),
   ('CATGATGTGACATTAAGATCAGC', 1),
   ('CATGCTGCCCGGAATGTGGAGGC', 

In [23]:
import json

with open('pai_patterns.json', 'w') as file:
    json.dump(unique_pai_patterns, file)
    
with open('cpai_patterns.json', 'w') as file:
    json.dump(unique_cpai_patterns, file)
    
with open('npai_patterns.json', 'w') as file:
    json.dump(unique_npai_patterns, file)

#### All islands

In [4]:
e_coli_data = read_data()

In [5]:
import logging

logging.basicConfig(filename='logs/tfidf_all_islands.log', level=logging.INFO,
                    format='%(message)s')

console = logging.StreamHandler()
console.setLevel(logging.INFO)
console.setFormatter(logging.Formatter('%(message)s'))

logging.getLogger('').addHandler(console)

In [1]:
from Bio import Align
from Bio.Seq import Seq

In [2]:
def count_matches(sequence, pattern):
    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'
    aligner.match_score = 2
    aligner.mismatch_score = -2
    aligner.open_gap_score = -2
    aligner.extend_gap_score = -1
    
    alignments = aligner.align(sequence, pattern)
    threshold = 0.85 * (2 * len(pattern))
    
    return sum(1 for alignment in alignments if alignment.score >= threshold)

In [8]:
def find_islands_kmers(full_sequence,
                       pai_islands,
                       cpai_islands,
                       npai_islands,
                       significant_kmers,
                       all_islands_kmers):
    
    all_islands = pai_islands + cpai_islands + npai_islands
    all_islands.sort(key=lambda x: x[0])
        
    for kmer, details in significant_kmers.items():
        
        if kmer in all_islands_kmers:
            continue
        
        start = 0
        end = 0
        isUnique = True
        
        # Check if kmer doesn't exist outside of the islands
        for island in all_islands:
            end = island[0]
            
            if end > start:
                if count_matches(full_sequence[start:end], kmer) > 0:
                    isUnique = False
                    break
                    
            start = island[1]
                    
        if not isUnique or count_matches(full_sequence[start:], kmer) > 0:
            continue
          
        
        # Check which islands contain kmer
        islands_containing_kmer = []
        
        for num, pai_island in enumerate(pai_islands, start=1):
            start = pai_island[0]
            end = pai_island[1]
            
            count = count_matches(full_sequence[start:end], kmer)
            
            if count > 0:
                islands_containing_kmer.append(f'PAI {num}: {count}')
            
        for num, cpai_island in enumerate(cpai_islands, start=1):
            start = cpai_island[0]
            end = cpai_island[1]
            
            count = count_matches(full_sequence[start:end], kmer)
            
            if count > 0:
                islands_containing_kmer.append(f'CPAI {num}: {count}')
                
        for num, npai_island in enumerate(npai_islands, start=1):
            start = npai_island[0]
            end = npai_island[1]
            
            count = count_matches(full_sequence[start:end], kmer)
            
            if count > 0:
                islands_containing_kmer.append(f'NPAI {num}: {count}')
        
        all_islands_kmers[kmer] = islands_containing_kmer

In [9]:
def get_islands_kmers(full_sequence,
                      island_sequence,
                      pai_islands,
                      cpai_islands,
                      npai_islands,
                      all_islands_kmers,
                      k_max):
    
    
    for k in range(4, k_max):
        tfidf_matrix, vectorizer = tfidf(island_sequence, k)
        kmers = get_kmers_map(tfidf_matrix, vectorizer, island_sequence)
        find_islands_kmers(full_sequence, pai_islands, cpai_islands, npai_islands, kmers, all_islands_kmers)

In [10]:
k_max = 100
logging.info(f'Max pattern length: {k_max}')
logging.info('--------------------------')

unique_patterns = {}

for i, (name, data) in enumerate(e_coli_data.items()):
    logging.info(f'Genome {i+1}/{len(e_coli_data)}: {name}')
    
    genome_sequence = data['genome_sequence']
    all_islands_kmers = {}

    pai_indices = [x[1] for x in data['pai_sequences']]
    cpai_indices = [x[1] for x in data['cpai_sequences']]
    npai_indices = [x[1] for x in data['npai_sequences']]
    
    if len(data['pai_sequences']) != 0:
        
        for j, (pai_sequence, indices) in enumerate(data['pai_sequences']):
            
            len_before = len(all_islands_kmers)
            
            get_islands_kmers(genome_sequence,
                              pai_sequence,
                              pai_indices,
                              cpai_indices,
                              npai_indices,
                              all_islands_kmers,
                              k_max)
            
            new_patterns_found = len(all_islands_kmers) - len_before
            logging.info(f"PAI {j+1}/{len(data['pai_sequences'])}: {new_patterns_found} patterns found")
                        
    else:
        logging.info('No PAI sequences')
    
    
    if len(data['cpai_sequences']) != 0:
        
        for j, (cpai_sequence, indices) in enumerate(data['cpai_sequences']):
            
            len_before = len(all_islands_kmers)
            
            get_islands_kmers(genome_sequence,
                              cpai_sequence,
                              pai_indices,
                              cpai_indices,
                              npai_indices,
                              all_islands_kmers,
                              k_max)
            
            new_patterns_found = len(all_islands_kmers) - len_before
            logging.info(f"CPAI {j+1}/{len(data['cpai_sequences'])}: {new_patterns_found} patterns found")
            
    else:
        logging.info('No CPAI sequences')
        
        
    if len(data['npai_sequences']) != 0:

        for j, (npai_sequence, indices) in enumerate(data['npai_sequences']):
            
            len_before = len(all_islands_kmers)
            
            get_islands_kmers(genome_sequence,
                              npai_sequence,
                              pai_indices,
                              cpai_indices,
                              npai_indices,
                              all_islands_kmers,
                              k_max)
            
            new_patterns_found = len(all_islands_kmers) - len_before
            logging.info(f"NPAI {j+1}/{len(data['npai_sequences'])}: {new_patterns_found} patterns found")
    else:
        logging.info('No NPAI sequences')
    logging.info('--------------------------')
    
    unique_patterns[name] = all_islands_kmers
    os.makedirs(f'e_coli_paidb/{name}/patterns/all_islands/', exist_ok=True)
    
    with open(f'e_coli_paidb/{name}/patterns/all_islands/tfidf.txt', 'w') as file:
        for kmer in all_islands_kmers:
            file.write(kmer)
            file.write('\n')
            
            for island in all_islands_kmers[kmer]:
                file.write(island)
                file.write('\n')
            
            file.write('----------------\n')

Max pattern length: 100
--------------------------
Genome 1/90: NC_013365
No PAI sequences
CPAI 1/1: 1801 patterns found
No NPAI sequences
--------------------------
Genome 2/90: NC_020163
No PAI sequences
CPAI 1/2: 1649 patterns found
CPAI 2/2: 1726 patterns found
NPAI 1/1: 1736 patterns found
--------------------------
Genome 3/90: NC_017635
No PAI sequences
CPAI 1/4: 863 patterns found
CPAI 2/4: 1749 patterns found
CPAI 3/4: 1610 patterns found
CPAI 4/4: 1742 patterns found
NPAI 1/2: 1742 patterns found
NPAI 2/2: 1739 patterns found
--------------------------
Genome 4/90: NC_012971
No PAI sequences
CPAI 1/7: 1742 patterns found
CPAI 2/7: 1766 patterns found
CPAI 3/7: 1750 patterns found
CPAI 4/7: 1742 patterns found
CPAI 5/7: 1757 patterns found
CPAI 6/7: 1488 patterns found
CPAI 7/7: 1753 patterns found
NPAI 1/1: 1688 patterns found
--------------------------
Genome 5/90: NC_017625
No PAI sequences
CPAI 1/3: 996 patterns found
CPAI 2/3: 1744 patterns found
CPAI 3/3: 1751 patterns f

CPAI 10/10: 1751 patterns found
NPAI 1/2: 1750 patterns found
NPAI 2/2: 1748 patterns found
--------------------------
Genome 29/90: NC_011750
No PAI sequences
CPAI 1/9: 1361 patterns found
CPAI 2/9: 1754 patterns found
CPAI 3/9: 1450 patterns found
CPAI 4/9: 1717 patterns found
CPAI 5/9: 1680 patterns found
CPAI 6/9: 1686 patterns found
CPAI 7/9: 1750 patterns found
CPAI 8/9: 906 patterns found
CPAI 9/9: 0 patterns found
NPAI 1/2: 1638 patterns found
NPAI 2/2: 1741 patterns found
--------------------------
Genome 30/90: NC_012967
No PAI sequences
CPAI 1/7: 1741 patterns found
CPAI 2/7: 1765 patterns found
CPAI 3/7: 1683 patterns found
CPAI 4/7: 1742 patterns found
CPAI 5/7: 1756 patterns found
CPAI 6/7: 1487 patterns found
CPAI 7/7: 1753 patterns found
NPAI 1/1: 1687 patterns found
--------------------------
Genome 31/90: NC_012947
No PAI sequences
CPAI 1/5: 1756 patterns found
CPAI 2/5: 1767 patterns found
CPAI 3/5: 1742 patterns found
CPAI 4/5: 1744 patterns found
CPAI 5/5: 1753 pat

NPAI 2/2: 1747 patterns found
--------------------------
Genome 66/90: NC_013941
No PAI sequences
CPAI 1/5: 704 patterns found
CPAI 2/5: 1757 patterns found
CPAI 3/5: 1749 patterns found
CPAI 4/5: 1714 patterns found
CPAI 5/5: 1750 patterns found
NPAI 1/2: 1751 patterns found
NPAI 2/2: 1741 patterns found
--------------------------
Genome 67/90: NC_017633
No PAI sequences
CPAI 1/7: 656 patterns found
CPAI 2/7: 1699 patterns found
CPAI 3/7: 1753 patterns found
CPAI 4/7: 1517 patterns found
CPAI 5/7: 1608 patterns found
CPAI 6/7: 1751 patterns found
CPAI 7/7: 1659 patterns found
NPAI 1/2: 1691 patterns found
NPAI 2/2: 1747 patterns found
--------------------------
Genome 68/90: NC_013361
PAI 1/1: 1744 patterns found
CPAI 1/6: 1070 patterns found
CPAI 2/6: 1749 patterns found
CPAI 3/6: 1748 patterns found
CPAI 4/6: 1714 patterns found
CPAI 5/6: 1543 patterns found
CPAI 6/6: 1334 patterns found
NPAI 1/1: 1713 patterns found
--------------------------
Genome 69/90: NC_017637
No PAI sequence

In [13]:
with open('all_islands.json', 'w') as file:
    json.dump(unique_patterns, file)

### NOSEP

In [3]:
from spmf import Spmf
import os
from Bio import SeqIO

In [4]:
def to_spmf_format(genome_sequence):
    seq = str(genome_sequence)
    
    seq = seq.replace('A', '1')
    seq = seq.replace('C', '2')
    seq = seq.replace('G', '3')
    seq = seq.replace('T', '4')

    seq = ''.join(c + ' -1 ' for c in seq).rstrip(' -1 ') + ' -2'
    
    return seq

In [5]:
def from_spmf_format(filename):
    patterns = []
    
    with open(filename, 'r') as file:
        for line in file:
            lst = line.split('#')
            
            pattern = lst[0]
            support = lst[1]
            
            pattern = pattern.replace('-1', '')
            pattern = pattern.replace('1', 'A')
            pattern = pattern.replace('2', 'C')
            pattern = pattern.replace('3', 'G')
            pattern = pattern.replace('4', 'T')
            pattern = pattern.replace(' ', '')
            
            support = int(support.replace('SUP: ', ''))
            patterns.append((pattern, support))
            
    return patterns

In [6]:
import logging

logging.basicConfig(filename='logs/nosep.log', level=logging.INFO,
                    format='%(message)s')

console = logging.StreamHandler()
console.setLevel(logging.INFO)
console.setFormatter(logging.Formatter('%(message)s'))

logging.getLogger('').addHandler(console)

In [7]:
e_coli_data = read_data()

In [8]:
def run_nosep(full_sequence, island_sequence, start_index, end_index, k_max):
    
    unique_patterns = []
    
    input_filename = 'input.txt'
    output_filename = 'output.txt'
    
    with open(input_filename, 'w') as input_file:
        input_file.write(to_spmf_format(island_sequence))
    
    spmf = Spmf("NOSEP", input_filename=input_filename, output_filename=output_filename, arguments=[4, k_max, 0, 0, 2])
    spmf.run()
    
    patterns = from_spmf_format(output_filename)
    
    os.remove(input_filename)
    os.remove(output_filename)
    
    for pattern, support in patterns:
        left_occurrences = full_sequence[:start_index].count(pattern)
        right_occurrences = full_sequence[end_index:].count(pattern)
        occurrences = left_occurrences + right_occurrences
        if occurrences == 0:
            unique_patterns.append((pattern, support))
            
    return unique_patterns

In [9]:
unique_pai_patterns  = {}
unique_cpai_patterns = {}
unique_npai_patterns = {}

k_max = 100
logging.info(f'Max pattern length: {k_max}')
logging.info('--------------------------')

for i, (name, data) in enumerate(e_coli_data.items()):
    logging.info(f'Genome {i+1}/{len(e_coli_data)}: {name}')
    
    if len(data['pai_sequences']) != 0:
        unique_pai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (pai_sequence, indices) in enumerate(data['pai_sequences']):
            
            if indices[1] - indices[0] > 40000:
                logging.info(f"PAI {j+1}/{len(data['pai_sequences'])}: island too large")
                continue
                
            unique_patterns = run_nosep(genome_sequence, pai_sequence, indices[0], indices[1], k_max)

            unique_pai_patterns[name]['pai_sequence'] = pai_sequence
            unique_pai_patterns[name]['indices'] = indices
            unique_pai_patterns[name]['patterns'] = unique_patterns

            logging.info(f"PAI {j+1}/{len(data['pai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'e_coli_paidb/{name}/patterns/nosep/pai/', exist_ok=True)
            
            with open(f'e_coli_paidb/{name}/patterns/nosep/pai/pai_{j}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
                        
    else:
        logging.info('No PAI sequences')
    
    if len(data['cpai_sequences']) != 0:    
        unique_cpai_patterns[name] = {}

        genome_sequence = data['genome_sequence']
        for j, (cpai_sequence, indices) in enumerate(data['cpai_sequences']):
            
            if indices[1] - indices[0] > 40000:
                logging.info(f"CPAI {j+1}/{len(data['cpai_sequences'])}: island too large")
                continue
            
            unique_patterns = run_nosep(genome_sequence, cpai_sequence, indices[0], indices[1], k_max)

            unique_cpai_patterns[name]['cpai_sequence'] = cpai_sequence
            unique_cpai_patterns[name]['indices'] = indices
            unique_cpai_patterns[name]['patterns'] = unique_patterns
            
            logging.info(f"CPAI {j+1}/{len(data['cpai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'e_coli_paidb/{name}/patterns/nosep/cpai/', exist_ok=True)
            
            with open(f'e_coli_paidb/{name}/patterns/nosep/cpai/cpai_{j}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
            
    else:
        logging.info('No CPAI sequences')
        
    if len(data['npai_sequences']) != 0:
        unique_npai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (npai_sequence, indices) in enumerate(data['npai_sequences']):
            
            if indices[1] - indices[0] > 40000:
                logging.info(f"NPAI {j+1}/{len(data['npai_sequences'])}: island too large")
                continue
            
            unique_patterns = run_nosep(genome_sequence, npai_sequence, indices[0], indices[1], k_max)

            unique_npai_patterns[name]['npai_sequence'] = npai_sequence
            unique_npai_patterns[name]['indices'] = indices
            unique_npai_patterns[name]['patterns'] = unique_patterns

            logging.info(f"NPAI {j+1}/{len(data['npai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'e_coli_paidb/{name}/patterns/nosep/npai/', exist_ok=True)
            
            with open(f'e_coli_paidb/{name}/patterns/nosep/npai/npai_{j}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
    
    else:
        logging.info('No NPAI sequences')
    logging.info('--------------------------')

Max pattern length: 100
--------------------------
Genome 1/90: NC_010468
No PAI sequences


>/home/veljko/.local/lib/python3.10/site-packages/spmf/spmf.jar
=============  NOSEP v2.46 - STATS =============
 Number of patterns found: 6452
 Total time ~ 13095 ms
 Maximum memory usage : 116.59021759033203 mb
 Calculation count 13811



CPAI 1/3: 4 unique patterns


>/home/veljko/.local/lib/python3.10/site-packages/spmf/spmf.jar
=============  NOSEP v2.46 - STATS =============
 Number of patterns found: 14619
 Total time ~ 69224 ms
 Maximum memory usage : 124.23494720458984 mb
 Calculation count 31685



CPAI 2/3: 70 unique patterns


...

CPAI 13/13: 181 unique patterns


>/home/veljko/.local/lib/python3.10/site-packages/spmf/spmf.jar
=============  NOSEP v2.46 - STATS =============
 Number of patterns found: 7121
 Total time ~ 15436 ms
 Maximum memory usage : 126.12447357177734 mb
 Calculation count 15654



NPAI 1/2: 3 unique patterns


>/home/veljko/.local/lib/python3.10/site-packages/spmf/spmf.jar
=============  NOSEP v2.46 - STATS =============
 Number of patterns found: 10541
 Total time ~ 35738 ms
 Maximum memory usage : 145.0367431640625 mb
 Calculation count 22813



NPAI 2/2: 24 unique patterns
--------------------------


In [18]:
with open('pai_patterns.json', 'w') as file:
    json.dump(unique_pai_patterns, file)
    
with open('cpai_patterns.json', 'w') as file:
    json.dump(unique_cpai_patterns, file)
    
with open('npai_patterns.json', 'w') as file:
    json.dump(unique_npai_patterns, file)

In [1]:
import os
import threading

In [2]:
all_patterns = {}

base_directory = './e_coli_paidb/'

for genome in os.listdir(base_directory):
    patterns_folder = os.path.join(base_directory, genome, 'patterns')

    if not os.path.isdir(patterns_folder):
        continue
        
    if genome not in all_patterns:
        all_patterns[genome] = {}
    
    nosep_folder = os.path.join(patterns_folder, 'nosep')
    
    nosep_pai_folder = os.path.join(nosep_folder, 'pai')
    nosep_cpai_folder = os.path.join(nosep_folder, 'cpai')
    nosep_npai_folder = os.path.join(nosep_folder, 'npai')
    
    if os.path.isdir(nosep_pai_folder):
        
        for filename in os.listdir(nosep_pai_folder):
            num = int((filename.split('_'))[1])
            pai_name = f'pai_{num+1}'
            
            if pai_name not in all_patterns[genome]:
                all_patterns[genome][pai_name] = {}
                
            with open(os.path.join(nosep_pai_folder, filename), 'r') as file:
                
                for line in file:
                    lst = line.split(':')
                    
                    pattern = lst[0]
                    frequency = int(lst[1])
                    
                    all_patterns[genome][pai_name][pattern] = frequency
                    
    if os.path.isdir(nosep_cpai_folder):
        
        for filename in os.listdir(nosep_cpai_folder):
            num = int((filename.split('_'))[1])
            cpai_name = f'cpai_{num+1}'
            
            if cpai_name not in all_patterns[genome]:
                all_patterns[genome][cpai_name] = {}
                
            with open(os.path.join(nosep_cpai_folder, filename), 'r') as file:
                
                for line in file:
                    lst = line.split(':')
                    
                    pattern = lst[0]
                    frequency = int(lst[1])
                    
                    all_patterns[genome][cpai_name][pattern] = frequency
                    
    if os.path.isdir(nosep_npai_folder):
        
        for filename in os.listdir(nosep_npai_folder):
            num = int((filename.split('_'))[1])
            npai_name = f'npai_{num+1}'
            
            if npai_name not in all_patterns[genome]:
                all_patterns[genome][npai_name] = {}
                
            with open(os.path.join(nosep_npai_folder, filename), 'r') as file:
                
                for line in file:
                    lst = line.split(':')
                    
                    pattern = lst[0]
                    frequency = int(lst[1])
                    
                    all_patterns[genome][npai_name][pattern] = frequency

In [3]:
base_directory = './e_coli_paidb/'

for genome in os.listdir(base_directory):
    patterns_folder = os.path.join(base_directory, genome, 'patterns')

    if not os.path.isdir(patterns_folder):
        continue
    
    if genome not in all_patterns:
        all_patterns[genome] = {}
    
    tfidf_folder = os.path.join(patterns_folder, 'tfidf')
    
    tfidf_pai_folder = os.path.join(tfidf_folder, 'pai')
    tfidf_cpai_folder = os.path.join(tfidf_folder, 'cpai')
    tfidf_npai_folder = os.path.join(tfidf_folder, 'npai')
    
    if os.path.isdir(tfidf_pai_folder):
        
        for filename in os.listdir(tfidf_pai_folder):
            num = int((filename.split('_'))[1])
            pai_name = f'pai_{num+1}'
            
            if pai_name not in all_patterns[genome]:
                all_patterns[genome][pai_name] = {}
                
            with open(os.path.join(tfidf_pai_folder, filename), 'r') as file:
                
                for line in file:
                    lst = line.split(':')
                    
                    pattern = lst[0]
                    frequency = int(lst[1])
                    
                    if pattern not in all_patterns[genome][pai_name]:
                        all_patterns[genome][pai_name][pattern] = frequency
                    
    if os.path.isdir(tfidf_cpai_folder):
        
        for filename in os.listdir(tfidf_cpai_folder):
            num = int((filename.split('_'))[1])
            cpai_name = f'cpai_{num+1}'
            
            if cpai_name not in all_patterns[genome]:
                all_patterns[genome][cpai_name] = {}
                
            with open(os.path.join(tfidf_cpai_folder, filename), 'r') as file:
                
                for line in file:
                    lst = line.split(':')
                    
                    pattern = lst[0]
                    frequency = int(lst[1])
                    
                    if pattern not in all_patterns[genome][cpai_name]:
                        all_patterns[genome][cpai_name][pattern] = frequency
                    
    if os.path.isdir(tfidf_npai_folder):
        
        for filename in os.listdir(tfidf_npai_folder):
            num = int((filename.split('_'))[1])
            npai_name = f'npai_{num+1}'
            
            if npai_name not in all_patterns[genome]:
                all_patterns[genome][npai_name] = {}
                
            with open(os.path.join(tfidf_npai_folder, filename), 'r') as file:
                
                for line in file:
                    lst = line.split(':')
                    
                    pattern = lst[0]
                    frequency = int(lst[1])
                    
                    if pattern not in all_patterns[genome][npai_name]:
                        all_patterns[genome][npai_name][pattern] = frequency

In [4]:
base_directory = './e_coli_paidb/'

for genome in os.listdir(base_directory):
    patterns_folder = os.path.join(base_directory, genome, 'patterns')

    if not os.path.isdir(patterns_folder):
        continue
    
    if genome not in all_patterns:
        all_patterns[genome] = {}
    
    tfidf_folder = os.path.join(patterns_folder, 'tfidf')
    all_islands_folder = os.path.join(tfidf_folder, 'all_islands')
    
    with open(os.path.join(all_islands_folder, 'tfidf.txt')) as file:
        pattern = None
        island_name = None
        
        for line in file:
            if line.startswith('-'):
                pattern = None
                
            elif 'PAI' in line:
                lst = line.split()
                island_name = lst[0].casefold() + '_' + lst[1].removesuffix(':')
                frequency = int(lst[2])
                
                if island_name not in all_patterns[genome]:
                    all_patterns[genome][island_name] = {}
                    
                if pattern not in all_patterns[genome][island_name]:
                    all_patterns[genome][island_name][pattern] = frequency
                    
            elif pattern is None:
                pattern = line.strip()
            
            else:
                print('Error')
                break

In [5]:
mutex = threading.Lock()

In [6]:
def find_recurring_patterns(genomes_lst, all_patterns, recurring_patterns, thread_num):
    
    for i, genome in enumerate(genomes_lst):
        for j, island_name in enumerate(all_patterns[genome]):
            for k, pattern in enumerate(all_patterns[genome][island_name]):
                
                with mutex:
                    if thread_num == 1:
                        print(f'Genome {i+1}/{len(genomes_lst)}  Island {j+1}/{len(all_patterns[genome])} Pattern {k+1}/{len(all_patterns[genome][island_name])}      ', end='\r')

                for other_genome in all_patterns:
                    if other_genome == genome:
                        continue

                    for other_island_name in all_patterns[other_genome]:

                        other_patterns = [key for key in all_patterns[other_genome][other_island_name] if len(key) == len(pattern)]

                        for other_pattern in other_patterns:

                            if other_pattern != pattern:
                                continue

                            if pattern not in recurring_patterns:
                                recurring_patterns[pattern] = {}

                            if genome not in recurring_patterns[pattern]:
                                recurring_patterns[pattern][genome] = {}

                            if island_name not in recurring_patterns[pattern][genome]:
                                recurring_patterns[pattern][genome][island_name] = all_patterns[genome][island_name][pattern]

                            if other_genome not in recurring_patterns[pattern]:
                                recurring_patterns[pattern][other_genome] = {}

                            if other_island_name not in recurring_patterns[pattern][other_genome]:
                                recurring_patterns[pattern][other_genome][other_island_name] = all_patterns[other_genome][other_island_name][pattern]

                            break

In [7]:
genomes_lst = sorted([genome for genome in all_patterns])
genomes_lst_fst = genomes_lst[:44]
len(genomes_lst_fst)

44

In [8]:
genomes_fst_1 = genomes_lst[:22]
genomes_fst_2 = genomes_lst[22:]

print(len(genomes_fst_1), len(genomes_fst_2))

22 22


In [9]:
recurring_patterns_fst_1 = {}
recurring_patterns_fst_2 = {}

In [10]:
thread1_fst = threading.Thread(target=find_recurring_patterns, args=(genomes_1, all_patterns, recurring_patterns_1, 1))
thread2_fst = threading.Thread(target=find_recurring_patterns, args=(genomes_2, all_patterns, recurring_patterns_2, 2))

thread1_fst.start()
thread2_fst.start()

thread1_fst.join()
thread2_fst.join()

In [11]:
recurring_patterns_fst_1

{'AGCCCCCACAG': {'NC_000913': {'cpai_4': 2},
  'NC_012971': {'cpai_6': 2},
  'NC_012892': {'cpai_6': 2},
  'NC_011745': {'cpai_11': 2},
  'NC_011751': {'cpai_7': 2},
  'NC_011742': {'cpai_7': 2},
  'NC_010473': {'cpai_4': 2},
  'NC_017638': {'cpai_3': 2},
  'NC_017626': {'cpai_11': 2}},
 'AGCGGGAACGT': {'NC_000913': {'cpai_4': 2},
  'NC_020163': {'cpai_1': 2},
  'NC_012971': {'cpai_6': 2},
  'NC_012892': {'cpai_6': 2},
  'NC_011745': {'cpai_11': 2},
  'NC_011751': {'cpai_7': 2},
  'NC_012967': {'cpai_6': 2},
  'NC_007779': {'cpai_4': 2},
  'NC_010468': {'cpai_1': 2},
  'NC_009801': {'cpai_6': 2},
  'NC_011742': {'cpai_7': 2},
  'NC_010473': {'cpai_4': 2},
  'NC_017638': {'cpai_3': 2}},
 'GTTAAGTACCT': {'NC_000913': {'cpai_4': 2},
  'NC_020163': {'cpai_1': 2},
  'NC_012971': {'cpai_6': 2},
  'NC_012892': {'cpai_6': 2},
  'NC_011745': {'cpai_11': 2},
  'NC_011751': {'cpai_7': 2},
  'NC_011750': {'cpai_9': 2},
  'NC_012967': {'cpai_6': 2},
  'NC_009801': {'cpai_6': 2},
  'NC_011742': {'cp

In [12]:
recurring_patterns_fst_2

{'CTAAGT': {'NC_011419': {'npai_1': 2}, 'NC_018659': {'cpai_1': 3}},
 'AAAGCTA': {'NC_011419': {'npai_1': 2},
  'NC_017665': {'npai_1': 2},
  'NC_017630': {'npai_1': 2},
  'NC_018659': {'cpai_1': 2},
  'NC_017637': {'npai_1': 2},
  'NC_017642': {'cpai_1': 2}},
 'AAGCTAC': {'NC_011419': {'npai_1': 2},
  'NC_017665': {'npai_1': 2},
  'NC_018659': {'cpai_1': 2},
  'NC_017637': {'npai_1': 2},
  'NC_017642': {'cpai_1': 2}},
 'AATAGAC': {'NC_011419': {'npai_1': 2},
  'NC_017665': {'npai_1': 2},
  'NC_017637': {'npai_1': 2},
  'NC_017642': {'cpai_1': 2}},
 'AATAGTT': {'NC_011419': {'npai_1': 2},
  'NC_018659': {'cpai_1': 2},
  'NC_017642': {'cpai_1': 2}},
 'AATTAAA': {'NC_011419': {'npai_1': 3}, 'NC_017642': {'cpai_1': 3}},
 'ACCAAGA': {'NC_011419': {'npai_1': 2},
  'NC_017665': {'npai_1': 3},
  'NC_018659': {'cpai_1': 3},
  'NC_017637': {'npai_1': 3},
  'NC_017642': {'cpai_1': 3}},
 'ACCCATA': {'NC_011419': {'npai_1': 3},
  'NC_017665': {'npai_1': 3},
  'NC_018659': {'cpai_1': 3},
  'NC_0176

In [13]:
import json

In [14]:
with open('recurring_patterns_00_44_1.json', 'w') as file:
    json.dump(recurring_patterns_fst_1, file)

In [15]:
with open('recurring_patterns_00_44_2.json', 'w') as file:
    json.dump(recurring_patterns_fst_2, file)